In [1]:
# Requirements: python=3.9+ numpy matplotlib jupyter ipython ipympl pylint ipywidgets nose
# Enabling the `widget` backend.
# This requires jupyter-matplotlib a.k.a. ipympl.
%matplotlib widget

import numpy as np
from utils import (State, ARAStar_Plotter, ARAStar_Planner,
                   GRAPH_LARGE, GRAPH_SMALL)
import tests

Add some intro about ARA*, how it works, what it's trying to solve, etc. Some graphics pulled from the slides or paper or online would be nice. Follow the recommendations on the advanced lecture assignment page. Include something about the effect changing epsilon has on the search

Add some instructions like 'our ARA* implementation is below, you'll need to fill out the following methods: initialize(), improve_path(), calc_epsilon_prime(), run() (include a brief description for each method). we've included testing code for each method so you can implement them one at a time, but the tests build on each other so make sure to go in order'.

Something like 'pseudocode for each of the methods you'll need to implement is included here. it's based on the pseudocode from the original ARA* paper, with some additions to make it easier to visualize and test our outputs.' And then include the pseudocode images, ideally one next to the other.

In [ ]:
def initialize(planner: ARAStar_Planner):
    raise NotImplementedError

In [4]:
tests.test_initalize(initialize)

In [ ]:
def improve_path(planner: ARAStar_Planner):
    raise NotImplementedError

In [ ]:
tests.test_improve_path(improve_path, initialize)

In [ ]:
def calc_epsilon_prime(planner: ARAStar_Planner):
    raise NotImplementedError

In [ ]:
tests.test_calc_epsilon_prime(calc_epsilon_prime, improve_path, initialize)

In [ ]:
def run(planner: ARAStar_Planner):
    raise NotImplementedError

In [ ]:
tests.test_run(run)

Now that we've verified our implementation, let's see how it works in action, by visualizing how the search progresses along a graph with a two different potential paths for different values of $\epsilon$

In [ ]:
EPSILON = 2.5
START = State(0, 0)
GOAL = State(6, 5)

planner = ARAStar_Planner(GRAPH_SMALL, START, GOAL, EPSILON, stepsize=1)
plotter = ARAStar_Plotter(GRAPH_SMALL, START, GOAL)
run(planner)

In [ ]:
plotter.plot_episode(2.5, planner.alg_history[2.5])

In [ ]:
plotter.plot_episode(1.5, planner.alg_history[1.5])

In [ ]:
plotter.plot_paths_found(planner.paths_found)

Add some notes about how the graphs should look like, explain how widget works

## Applying ARA* to Larger Graphs with Multiple Branches (10 pts)

Now lets try applying our planner to larger regions where there might be many different paths we can take.

In [ ]:
EPSILON = 1.5
START = State(24, 4)
GOAL = State(4, 44)

planner = ARAStar_Planner(GRAPH_LARGE, START, GOAL, EPSILON, stepsize=0.2)
plotter = ARAStar_Plotter(GRAPH_LARGE, START, GOAL)
run(planner)

In [ ]:
plotter.plot_paths_found(planner.paths_found)

Why does the graph look like this? How does eps changing affect the path?

**Your answer here**